In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.model_selection import GroupShuffleSplit

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import callbacks

In [2]:
#Set Relative Path
import sys
sys.path.append("../src")

In [3]:
# Import data
df_dataK = pd.read_csv(r'../data/d03_cleaned_data/datall.csv')
df_dataK.head()

,text,emotion
0,i got up feeling horny this morning,love
1,i will go to my mailbox and talk to the mailma...,happy
2,i understand that any of my extremely positive...,happy
3,i suggest you give it a listen i feel like i a...,love
4,i have been with petronas for years i feel tha...,happy


In [4]:
output_shape = len(df_dataK['emotion'].unique())

In [5]:
y=df_dataK['emotion']
X=df_dataK['text']

In [6]:
input_shape = [1]

In [ ]:
model = keras.Sequential([
    layers.Dense(1024, activation='relu', input_shape=input_shape),
    layers.Dropout(0.3),
    layers.BatchNormalization(),
    layers.Dense(1024, activation='relu'),
    layers.Dropout(0.3),
    layers.BatchNormalization(),
    layers.Dense(1024, activation='relu'),
    layers.Dropout(0.3),
    layers.BatchNormalization(),
    layers.Dense(1),
])

In [ ]:
    
model.compile(
    optimizer='adam', # SGD is more sensitive to differences of scale
    loss="categorical_crossentropy",
    metrics=["categorical_accuracy"],
)
history = model.fit(
    X, y,
    validation_split=0.2,
    batch_size=64,
    epochs=100,
    verbose=0,
)

In [ ]:
X.values

In [ ]:
import autokeras as ak
from tensorflow import keras

clf = ak.TextClassifier(objective='accuracy', loss='categorical_crossentropy', metrics='accuracy', overwrite=True, max_trials=10)

early_stopping = keras.callbacks.EarlyStopping(
    patience=10,
    min_delta=0.001,
    restore_best_weights=True,
)

clf = clf.fit(
    X.values, y,
    validation_split=0.2,
    callbacks=[early_stopping],
    verbose=1, # hide the output because we have so many epochs
)

In [7]:
import tensorflow_hub as hub

module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1"
#module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"

bert_layer = hub.KerasLayer(module_url, trainable=True)

In [8]:
!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py

In [9]:
!pip install sentencepiece &> /dev/null

In [10]:
import tokenization
import numpy as np

def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [11]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

In [70]:
X = bert_encode(df_dataK.text.values, tokenizer, max_len=100)
y = df_dataK.emotion.values

In [25]:
X

(array([[ 101, 1045, 2288, ...,    0,    0,    0],
        [ 101, 1045, 2097, ...,    0,    0,    0],
        [ 101, 1045, 3305, ...,    0,    0,    0],
        ...,
        [ 101, 3407, 2388, ...,    0,    0,    0],
        [ 101, 1030, 9152, ...,    0,    0,    0],
        [ 101, 1030, 9587, ...,    0,    0,    0]]),
 array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]]),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]))

In [28]:
input_shape = [len(X)]

In [31]:
len(X)

3

In [32]:
len(X[0])

47629

In [36]:
len(X[0][0])

100

In [49]:
model2 = keras.Sequential()
model2.add(layers.Dense(3, activation='relu', input_shape=[3]))
model2.output_shape

(None, 3)

In [50]:
model2.add(layers.Flatten())
model2.output_shape

(None, 3)

In [58]:
model = keras.Sequential([
    layers.Dense(3, activation='relu', input_shape=[3]),
#     layers.Dense(3, activation='relu'),
    layers.Dropout(0.3),
    layers.BatchNormalization(),
    layers.Dense(1024, activation='relu'),
    layers.Dropout(0.3),
    layers.BatchNormalization(),
    layers.Dense(1024, activation='relu'),
    layers.Dropout(0.3),
    layers.BatchNormalization(),
    layers.Dense(1),
])

In [59]:
model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_26 (Dense)             (None, 3)                 12        
_________________________________________________________________
dropout_15 (Dropout)         (None, 3)                 0         
_________________________________________________________________
batch_normalization_15 (Batc (None, 3)                 12        
_________________________________________________________________
dense_27 (Dense)             (None, 1024)              4096      
_________________________________________________________________
dropout_16 (Dropout)         (None, 1024)              0         
_________________________________________________________________
batch_normalization_16 (Batc (None, 1024)              4096      
_________________________________________________________________
dense_28 (Dense)             (None, 1024)            

In [60]:
    
model.compile(
    optimizer='adam', # SGD is more sensitive to differences of scale
    loss="categorical_crossentropy",
    metrics=["categorical_accuracy"],
)
history = model.fit(
    X, y,
    validation_split=0.2,
    batch_size=64,
    epochs=100,
    verbose=1,
)

Epoch 1/100


ValueError: in user code:

    /home/apprenant/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    /home/apprenant/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/apprenant/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/apprenant/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/apprenant/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/apprenant/anaconda3/lib/python3.8/site-pack

In [84]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint

def build_model(bert_layer, max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    _, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :]
    out = Dense(6, activation='softmax')(clf_output)
    
    model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(Adam(lr=2e-6), loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [85]:
model = build_model(bert_layer, max_len=100)

In [86]:
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 100)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 100)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 100)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 768), (None, 109482241   input_word_ids[0][0]             
                                                                 input_mask[0][0]           

In [87]:
X[0].shape[1]

100

In [88]:
from sklearn.preprocessing import OneHotEncoder


In [89]:
enc = OneHotEncoder(handle_unknown='ignore')

In [90]:
ydum = pd.get_dummies(df_dataK.emotion)

In [91]:
ydum.to_numpy()

array([[0, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       ...,
       [0, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0, 0]], dtype=uint8)

In [ ]:
train_history = model.fit(
    X, ydum,
    validation_split=0.2,
    epochs=5,
    batch_size=32
)

Epoch 1/5
1191/1191 [==============================] - 10796s 9s/step - loss: 1.4865 - accuracy: 0.3966 - val_loss: 1.3062 - val_accuracy: 0.4970
Epoch 2/5
1191/1191 [==============================] - 10772s 9s/step - loss: 0.9530 - accuracy: 0.6346 - val_loss: 1.2700 - val_accuracy: 0.5267
Epoch 3/5
 206/1191 [====>.........................] - ETA: 2:27:23 - loss: 0.8081 - accuracy: 0.6879